In [1]:
pwd

u'/Users/naveenkumar/Dropbox/OnlineWork/Notebooks'

In [4]:
%matplotlib inline
import pandas as pd
import numpy as np
from datetime import datetime
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import warnings
warnings.filterwarnings('ignore')

fdir = '/Users/naveenkumar/Dropbox/OnlineWork/Datasets/McKinseyData/'



In [5]:
train_data = pd.read_csv(fdir + 'train.csv')
test_data  = pd.read_csv(fdir + 'test.csv' )
full_data = [train_data, test_data]

In [6]:
def columntype(x):
    t = ''
    if is_numeric_dtype(x) == True:
        t = 'numeric'
    else:
        t = 'string'
    
    return t

def unique_value_counts(x,y,cut_off):
    if y <= cut_off:
        return (s2[x].value_counts()).to_dict()
    
    
def replace_DDMMYY_WITH_DDMMYYYY(indate, AddCentury):
    if len(str(indate)) == 8:
        YY = str(indate)[6:8]
        YY_INT = int(YY) + AddCentury
        return str(indate)[0:6] + str(YY_INT)
    else:
        return np.nan
    

def datediff(d1,d2,date_format):
    df = date_format
    a = datetime.strptime(str(d1), df)
    b = datetime.strptime(str(d2), df)
    delta = b - a
    return delta.days

def percentile_dist(x):
    print("-----------------------------------")
    for t in np.arange(0.01,1.0,0.01):
        print(str(t) + " : " + str(x.quantile(q=t)))

        
        
def ds_summary(s2):
    cmd = pd.DataFrame(s2.columns,columns=['column_name'])
    cmd['TotalRows'] = len(s2)
    cmd['TotalUniqueLevels'] = cmd['column_name'].apply(lambda col:s2[col].nunique())
    cmd['PercentUniqueValues'] = cmd['TotalUniqueLevels']/cmd['TotalRows']
    cmd['TotalNullValues'] = cmd['column_name'].apply(lambda col:s2[col].isnull().sum())
    cmd['PercentNullValues'] = cmd['TotalNullValues']/cmd['TotalRows']
    cmd['IdColFlag'] = cmd['TotalRows'] == cmd['TotalUniqueLevels']
    cmd['DataType'] = cmd['column_name'].apply(lambda col:columntype(s2[col]))
    cmd['UniqueValue'] = cmd[cmd['TotalUniqueLevels'] < 11]['column_name'].apply(lambda col:(s2[col].value_counts()).to_dict())
    cmd['mean'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(s2[col].mean(),2))
    cmd['min'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(s2[col].min(),2))
    cmd['q1'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(s2[col].quantile(q=0.25),2))
    cmd['median'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(s2[col].median(),2))
    cmd['q3'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(s2[col].quantile(q=0.75),2))
    cmd['max'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(s2[col].max(),2))
    return cmd


In [7]:
col_summary = ds_summary(train_data)
col_summary.to_csv(fdir + 'train_col_summary.psv', sep = "|")

In [8]:
col_list = [
 'Gender'
,'Lead_Creation_Date'
,'Monthly_Income'
,'Contacted'
,'Source_Category'
,'Var1'
,'Approved'
,'DOB'
,'Existing_EMI'
,'City_Category'
,'Employer_Category1'
,'Employer_Category2'
,'Primary_Bank_Type'
,'Loan_Amount'
,'Loan_Period'
,'Interest_Rate'
,'EMI']

In [9]:

tr = train_data[col_list]
tr['LCD_NEW'] = tr['Lead_Creation_Date'].apply(lambda x:replace_DDMMYY_WITH_DDMMYYYY(x,2000))
tr['DOB_NEW'] = tr['DOB'].apply(lambda x:replace_DDMMYY_WITH_DDMMYYYY(x,1900))
tr["DOB_NEW"].fillna('11/01/1982', inplace=True)
tr['LCD_DAYS'] = tr['LCD_NEW'].apply(lambda x: datediff(x,'31/01/2018',"%d/%m/%Y"))
tr['AGE_YEAR'] = tr['DOB_NEW'].apply(lambda x: round((datediff(x,'31/01/2018',"%d/%m/%Y")*1.0)/365,0))

In [10]:
tr.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Monthly_Income,69713.0,5622.283200,174767.061840,0.00,1650.00,2500.0,4000.0,38383838.3
Var1,69713.0,3.948446,3.819214,0.00,0.00,2.0,7.0,10.0
Approved,69713.0,0.014631,0.120073,0.00,0.00,0.0,0.0,1.0
Existing_EMI,69662.0,360.928751,2288.517927,0.00,0.00,0.0,350.0,545436.5
Employer_Category2,65415.0,3.720187,0.807374,1.00,4.00,4.0,4.0,4.0
Loan_Amount,42004.0,39429.982859,30727.595990,5000.00,20000.00,30000.0,50000.0,300000.0
Loan_Period,42004.0,3.890629,1.167491,1.00,3.00,4.0,5.0,6.0
Interest_Rate,22276.0,19.213570,5.847136,11.99,15.25,18.0,20.0,37.0
EMI,22276.0,1101.466242,752.661394,118.00,649.00,941.0,1295.0,13556.0
LCD_DAYS,69713.0,531.110926,26.931771,488.00,507.00,530.0,554.0,579.0


In [11]:
print(tr['City_Category'].value_counts())
print(tr['Employer_Category1'].value_counts())
print(tr['Employer_Category2'].value_counts())
print(tr['Primary_Bank_Type'].value_counts())

A    49885
C    11694
B     7320
Name: City_Category, dtype: int64
A    33336
B    18056
C    14303
Name: Employer_Category1, dtype: int64
4.0    57582
1.0     4258
2.0     1955
3.0     1620
Name: Employer_Category2, dtype: int64
P    39619
G    20703
Name: Primary_Bank_Type, dtype: int64


In [184]:

tr["I_Primary_Bank_Type"] = tr["Primary_Bank_Type"]
tr["I_Primary_Bank_Type"].fillna('M', inplace=True) #New value as proportion is 13%

tr["I_Employer_Category1"] = tr["Employer_Category1"]
tr["I_Employer_Category1"].fillna('M', inplace=True)#New value as proportion is 6%

tr["I_Employer_Category2"] = tr["Employer_Category2"]
tr["I_Employer_Category2"].fillna(5, inplace=True)#New value as proportion is 6%

tr["I_City_Category"] = tr["City_Category"]
tr["I_City_Category"].fillna('A', inplace=True) #Existing value as proportion is 1%


In [185]:
tr['I_Loan_Amount'] = tr['Loan_Amount']
tr['I_Loan_Period'] = tr['Loan_Period']
tr['I_Interest_Rate'] = tr['Interest_Rate']
tr['I_EMI'] = tr['EMI']
tr['I_Existing_EMI'] = tr['Existing_EMI']

tr['I_Loan_Amount'].fillna(0, inplace = True)
tr['I_Loan_Period'].fillna(0, inplace = True)
tr['I_Interest_Rate'].fillna(0, inplace = True)
tr['I_EMI'].fillna(0, inplace = True)
tr['I_Existing_EMI'].fillna(0, inplace = True)

In [186]:
col_summary = ds_summary(tr)
col_summary.to_csv('/Users/naveenkumar/Desktop/sklearnNotebooks/train_col_summary.psv', sep = "|")

In [187]:
tr.columns

Index([u'Gender', u'Lead_Creation_Date', u'Monthly_Income', u'Contacted',
       u'Source_Category', u'Var1', u'Approved', u'DOB', u'Existing_EMI',
       u'City_Category', u'Employer_Category1', u'Employer_Category2',
       u'Primary_Bank_Type', u'Loan_Amount', u'Loan_Period', u'Interest_Rate',
       u'EMI', u'LCD_NEW', u'DOB_NEW', u'LCD_DAYS', u'AGE_YEAR',
       u'I_Primary_Bank_Type', u'I_Employer_Category1',
       u'I_Employer_Category2', u'I_City_Category', u'I_Loan_Amount',
       u'I_Loan_Period', u'I_Interest_Rate', u'I_EMI', u'I_Existing_EMI'],
      dtype='object')

In [188]:
col_list_model = [
'Gender',
'Monthly_Income',
'Contacted',
'Source_Category',
'Var1',
'Approved',
'LCD_DAYS',
'AGE_YEAR',
'I_Primary_Bank_Type',
'I_Employer_Category1',
'I_Employer_Category2',
'I_City_Category',
'I_Loan_Amount',
'I_Loan_Period',
'I_Interest_Rate',
'I_EMI',
'I_Existing_EMI']



In [222]:
ds = tr[col_list_model]


from sklearn import preprocessing
le = preprocessing.LabelEncoder()
ds['Contacted_dummy'] = le.fit_transform(ds['Contacted'])
ds['Gender_dummy'] = le.fit_transform(ds['Gender'])
ds['I_City_Category_dummy'] = le.fit_transform(ds['I_City_Category'])

ds['I_Primary_Bank_Type_dummy'] = le.fit_transform(ds['I_Primary_Bank_Type'])
ds['I_Employer_Category1_dummy'] = le.fit_transform(ds['I_Employer_Category1'])
ds['I_Employer_Category2_dummy'] = le.fit_transform(ds['I_Employer_Category2'])
ds['Var1_dummy'] = le.fit_transform(ds['Var1'])
ds['I_Loan_Period_dummy'] = le.fit_transform(ds['I_Loan_Period'])
ds['Source_Category_dummy'] = le.fit_transform(ds['Source_Category'])


ohe = preprocessing.OneHotEncoder()
dummy1 = pd.DataFrame(ohe.fit_transform(ds['I_City_Category_dummy'].reshape(-1,1)).toarray(), columns = ["City_CategoryA","City_CategoryB","City_CategoryC"])
dummy2 = pd.DataFrame(ohe.fit_transform(ds['I_Primary_Bank_Type_dummy'].reshape(-1,1)).toarray(), columns = ["I_Primary_Bank_Type_dummyG","I_Primary_Bank_Type_dummyM","I_Primary_Bank_Type_dummyP"])
dummy3 = pd.DataFrame(ohe.fit_transform(ds['I_Employer_Category1_dummy'].reshape(-1,1)).toarray(), columns = ["I_Employer_Category1_dummyA","I_Employer_Category1_dummyB","I_Employer_Category1_dummyC","I_Employer_Category1_dummyM"])
dummy4 = pd.DataFrame(ohe.fit_transform(ds['I_Employer_Category2_dummy'].reshape(-1,1)).toarray(), columns = ["I_Employer_Category2_dummy1","I_Employer_Category2_dummy2","I_Employer_Category2_dummy3","I_Employer_Category2_dummy4","I_Employer_Category2_dummy5"])
dummy5 = pd.DataFrame(ohe.fit_transform(ds['Var1_dummy'].reshape(-1,1)).toarray(), columns = ["Var1_dummy0","Var1_dummy2","Var1_dummy4","Var1_dummy7","Var1_dummy10"])
dummy6 = pd.DataFrame(ohe.fit_transform(ds['I_Loan_Period_dummy'].reshape(-1,1)).toarray(), columns = ["I_Loan_Period_dummy0","I_Loan_Period_dummy1","I_Loan_Period_dummy2","I_Loan_Period_dummy3","I_Loan_Period_dummy4","I_Loan_Period_dummy5","I_Loan_Period_dummy6"])
dummy7 = pd.DataFrame(ohe.fit_transform(ds['Source_Category_dummy'].reshape(-1,1)).toarray(), columns = ["Source_Category_dummyA","Source_Category_dummyB","Source_Category_dummyC","Source_Category_dummyD","Source_Category_dummyE","Source_Category_dummyF","Source_Category_dummyG"])


ds = pd.concat([ds, dummy1], axis = 1)
ds = pd.concat([ds, dummy2], axis = 1)
ds = pd.concat([ds, dummy3], axis = 1)
ds = pd.concat([ds, dummy4], axis = 1)
ds = pd.concat([ds, dummy5], axis = 1)
ds = pd.concat([ds, dummy6], axis = 1)
ds = pd.concat([ds, dummy7], axis = 1)

ds.head(10)

/Users/naveenkumar/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/naveenkumar/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/naveenkumar/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://panda

,Gender,Monthly_Income,Contacted,Source_Category,Var1,Approved,LCD_DAYS,AGE_YEAR,I_Primary_Bank_Type,I_Employer_Category1,...,I_Loan_Period_dummy4,I_Loan_Period_dummy5,I_Loan_Period_dummy6,Source_Category_dummyA,Source_Category_dummyB,Source_Category_dummyC,Source_Category_dummyD,Source_Category_dummyE,Source_Category_dummyF,Source_Category_dummyG
0,Female,2000.0,N,G,0,0,565,39.0,P,A,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Male,3500.0,Y,G,10,0,576,31.0,P,C,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,Male,2250.0,Y,B,0,0,561,35.0,G,C,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,Male,3500.0,Y,B,7,0,571,29.0,G,A,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,Male,10000.0,Y,B,10,0,560,33.0,P,A,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,Female,7000.0,N,B,0,0,579,29.0,P,A,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,Male,7500.0,Y,C,10,0,578,45.0,G,B,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,Female,3000.0,Y,B,0,0,577,27.0,P,A,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,Male,2500.0,Y,B,7,0,578,41.0,G,A,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,Female,2500.0,N,B,0,0,578,27.0,M,A,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [223]:
col_summary = ds_summary(ds)
col_summary.to_csv('/Users/naveenkumar/Desktop/sklearnNotebooks/train_col_summary.psv', sep = "|")

In [346]:
x_col = [
'Monthly_Income',
'LCD_DAYS',
'AGE_YEAR',
'I_Loan_Amount',
'I_Interest_Rate',
'I_EMI',
'I_Existing_EMI',
'Contacted_dummy',
'Gender_dummy',
'City_CategoryA',
'City_CategoryB',
#'City_CategoryC',
'I_Primary_Bank_Type_dummyG',
#'I_Primary_Bank_Type_dummyM',
'I_Primary_Bank_Type_dummyP',
'I_Employer_Category1_dummyA',
'I_Employer_Category1_dummyB',
'I_Employer_Category1_dummyC',
#'I_Employer_Category1_dummyM',
'I_Employer_Category2_dummy1',
'I_Employer_Category2_dummy2',
'I_Employer_Category2_dummy3',
'I_Employer_Category2_dummy4',
#'I_Employer_Category2_dummy5',
'Var1_dummy0',
'Var1_dummy2',
'Var1_dummy4',
'Var1_dummy7',
#'Var1_dummy10',
'I_Loan_Period_dummy0',
'I_Loan_Period_dummy1',
'I_Loan_Period_dummy2',
'I_Loan_Period_dummy3',
'I_Loan_Period_dummy4',
'I_Loan_Period_dummy5',
#'I_Loan_Period_dummy6',
'Source_Category_dummyA',
'Source_Category_dummyB',
'Source_Category_dummyC',
'Source_Category_dummyD',
'Source_Category_dummyE',
#'Source_Category_dummyF',
'Source_Category_dummyG']

y_col = 'Approved'

In [347]:
x = ds[x_col]
y = ds[y_col]

In [329]:
import sklearn 
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import roc_auc_score as ras 
from sklearn.metrics import precision_score as pre
from sklearn.metrics import recall_score as rec
from sklearn.metrics import f1_score as fsc
from sklearn.metrics import confusion_matrix as cfm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
def pm(y_a, y_p):
    print('acc : ' + str(acc(y_a, y_p)))
    print('pre : ' + str(pre(y_a, y_p)))
    print('rec : ' + str(rec(y_a, y_p)))
    print('fsc : ' + str(fsc(y_a, y_p)))
    print('auc : ' + str(ras(y_a, y_p)))
    print('------confusion matrix------')
    print(cfm(y_a, y_p))

In [340]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size  = 0.2, random_state = 42 )

In [312]:
for i in np.arange(1,50,1):
    rf = RandomForestClassifier(max_depth=i, random_state=42)
    rf.fit(x_train, y_train)
    pred_train = rf.predict(x_train)
    pred_test = rf.predict(x_test)
    print('AUC for MaxDepth '+ str(i) + ' tr :'+ str(round(ras(y_train, pred_train),2)) + ' ts :'+ str(round(ras(y_test, pred_test),2)))       
    #print('AUC for total tree of '+ str(i) + ' tr :'+ str(round(ras(y_train, pred_train),2)) + ' ts :'+ str(round(ras(y_test, pred_test),2)))           

AUC for MaxDepth 1 tr :0.5 ts :0.5
AUC for MaxDepth 2 tr :0.5 ts :0.5
AUC for MaxDepth 3 tr :0.5 ts :0.5
AUC for MaxDepth 4 tr :0.5 ts :0.5
AUC for MaxDepth 5 tr :0.5 ts :0.5
AUC for MaxDepth 6 tr :0.5 ts :0.5
AUC for MaxDepth 7 tr :0.5 ts :0.5
AUC for MaxDepth 8 tr :0.5 ts :0.5
AUC for MaxDepth 9 tr :0.5 ts :0.5


KeyboardInterrupt: 

In [318]:
rf = RandomForestClassifier(max_depth=20, random_state=42)
rf.fit(x_train, y_train)
pred_train = rf.predict(x_train)
#pred_test = rf.predict(x_test)
pm(y_train, pred_train)
#pm(y_test, pred_test)

acc : 0.993702752715
pre : 0.998284734134
rec : 0.570588235294
fsc : 0.726138490331
auc : 0.785286838885
------confusion matrix------
[[68692     1]
 [  438   582]]


In [348]:
for i in [500,1000]:
    lr = LogisticRegression(max_iter =i )
    lr.fit(x_train, y_train)
    pred_train = lr.predict(x_train)
    print('AUC for max_iter '+ str(i) + ' tr :'+ str(round(ras(y_train, pred_train),2)))


AUC for max_iter 500 tr :0.5
AUC for max_iter 1000 tr :0.5


In [349]:
lr = LogisticRegression()
lr.fit(x_train, y_train)
pred_train = lr.predict(x_train)
#pred_test = lr.predict(x_test)
pm(y_train, pred_train)
#pm(y_test, pred_test)



acc : 0.98527882374
pre : 0.111111111111
rec : 0.0012285012285
fsc : 0.00243013365735
auc : 0.500541465113
------confusion matrix------
[[54948     8]
 [  813     1]]


In [325]:
lr = LogisticRegression()
lr

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

acc : 0.996324188632
pre : 0.998363338789
rec : 0.749385749386
fsc : 0.856140350877
auc : 0.874683776505
------confusion matrix------
[[54955     1]
 [  204   610]]
acc : 0.985010399484
pre : 0.2
rec : 0.00485436893204
fsc : 0.00947867298578
auc : 0.50228159227
------confusion matrix------
[[13733     4]
 [  205     1]]


In [ ]:
def t2(x):
    y = x + '_dummy'
    print('-----------------------')
    print ds.groupby([x,y]).size()
    
def t1(x):
    print('-----------------------')
    print ds[x].value_counts()

t2('Contacted')
t2('Gender')
t2('I_City_Category')
t2('I_Primary_Bank_Type')
t2('I_Employer_Category1')
t2('I_Employer_Category2')
t2('Var1')
t2('I_Loan_Period')
t2('Source_Category')

In [335]:
gbc = GradientBoostingClassifier()
gbc.fit(x_train, y_train)
pred_train = gbc.predict(x_train)
#pred_test = gbc.predict(x_test)
pm(y_train, pred_train)
#pm(y_test, pred_test)

GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

acc : 0.985641128627
pre : 0.952380952381
rec : 0.0196078431373
fsc : 0.0384245917387
auc : 0.509796642807
------confusion matrix------
[[68692     1]
 [ 1000    20]]


In [344]:
from sklearn.ensemble import AdaBoostClassifier

abc = AdaBoostClassifier(RandomForestClassifier(max_depth = 15), n_estimators=20)
abc.fit(x_train, y_train)
pred_train = abc.predict(x_train)
pred_test = gbc.predict(x_test)
pm(y_train, pred_train)
pm(y_test, pred_test)

acc : 0.999802761341
pre : 0.997521685254
rec : 0.988943488943
fsc : 0.993214065392
auc : 0.994453548096
------confusion matrix------
[[54954     2]
 [    9   805]]
acc : 0.985369002367
pre : 1.0
rec : 0.00970873786408
fsc : 0.0192307692308
auc : 0.504854368932
------confusion matrix------
[[13737     0]
 [  204     2]]


In [345]:
abc.feature_importances_

array([  1.41723860e-01,   1.44776465e-01,   1.16031274e-01,
         7.46265199e-02,   4.29205956e-02,   4.98466489e-02,
         9.83917212e-02,   5.92672827e-03,   1.34396575e-02,
         1.16962931e-02,   5.12794463e-03,   5.64613154e-03,
         1.39896929e-02,   6.22309337e-03,   1.55879715e-02,
         1.85487037e-02,   1.78517803e-02,   1.78337842e-02,
         6.08152644e-03,   9.46439282e-03,   5.01096347e-03,
         2.57739911e-03,   1.53076886e-02,   5.24845977e-03,
         7.79980320e-03,   4.04296632e-03,   5.16307022e-03,
         1.38420345e-02,   1.66850958e-02,   5.42510166e-03,
         4.83734839e-03,   7.86093499e-03,   1.37184576e-02,
         1.25123045e-02,   1.63567776e-02,   4.71004636e-06,
         3.70032420e-09,   1.60696330e-02,   1.06123878e-02,
         6.27751275e-05,   7.24581583e-03,   8.54523368e-04,
         1.30269612e-02])